# Chapter 12 강화학습을 활용한 자연어 생성

## 12.1 들어가며

- 폴리시 그래디언트(policy gradient)

### 12.1.1 생성적 적대 신경망(GAN)

- 생성자 G
- 판별자 D
- 두 모델이 균형을 이루며 학습

$$\min_G \max_D \mathcal L(D,G) = \mathbb E_{x\sim p_r(x)}[logD(x)] + \mathbb E_{z\sim p_z(z)}[log(1-D(G(z))))]$$

#### GAN이 중요한 이유

생성된 이미지와 정답 이미지 간의 차이를 비교하는데 MSE를 사용하게 된다면 확률 분포의 중간으로 출력을 낼 수 밖에 없다. 따라서 흐릿한 이미지가 생성되는데 GAN은 더욱 정교한 목적함수를 D가 근사하여 해결한다.

### 12.1.2. GAN을 자연어 생성에 적용해보기

GAN을 seq2seq에 적용하려면 Argmax 또는 샘플링을 통해 원핫 벡터를 얻어야되는데 이 부분에서 역전파 수행이 어려워 학습이 불가능하다.

### 12.1.3 GAN과 자연어 생성

단어 또는 문장은 불연속적인 값이고 언어는 이런 불연속적인 값의 순차적인 배열이다. 신경망 언어 모델이나 seq2seq를 통해 잠재 공간에서는 연속적인 변수로 치환해서 다루지만, 실제 언어를 표현하기 위해서는 이산 확률 분포에서 샘플링하는 과정이 필요하다. 이러한 이유 때문에 D의 손실을 G에 전달할 수 없었다. 하지만 강화학습을 통해 적대적 학습 방식을 우회적으로 사용할 수 있게 되었다.

### 12.1.4 강화학습을 사용하는 이유

교차 엔트로피나 MSE 등으로 정의할 수 없는 목적함수를 잘 설계된 보상을 사용해서 해결 가능하기 때문이다.

## 12.2 강화학습 기초

### 12.2.1 유니버스

강화학습이란 어떤 객체가 주어진 환경에서, 상황에 따라 어떻게 행동해야 할지 학습하는 방식이다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile3.uf.tistory.com%2Fimage%2F99B1D7485ACF62550E6AE5" width="60%"/>

처음 상태인 $S_t$를 받고, 에이전트는 자신의 정책에 따라 행동 $A_t$를 선택한다. 환경은 에이전트로부터 선택된 행동을 받아 보상 $R_{t+1}$과 새롭게 바뀐 상태 $S_{t+1}$을 반환한다. 에이전트는 다시 새로운 상태를 받아 다음 행동을 선택한다. 특정 조건이 만족되면 환경은 시퀀스를 종료하는데 이를 하나의 에피소드(episode)라고 한다. 에이전트가 반복되는 에피소드에서 보상을 최대화하는 방향으로 학습하도록 하는 것이 목표이다.

### 12.2.2 마르코프 결정 과정

- 현재 $T=t$, 미래 $T>t$, 과거 $T<t$
- 현재 상황에서 미래 상황으로 바뀔 확률 $P(S'|S)$
- 마르코프 결정 과정은 현재 어떤 행동을 했을 때 미래 상황으로 바뀔 확률 $P(S'|S,A)$

### 12.2.3 보상

- 어떤 시점부터 받은 보상의 누적 합 $G_t \doteq R_{t+1} + R_{t+2} + R_{t+3} + \dots + R_T$
- 감소율($\gamma$) 도입 $G_t \doteq R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \dots = \sum_{k=0}^\infty \gamma^k R_{t+k+1}$

### 12.2.4 정책

앞으로 받을 보상의 누적 합이 최대가 되도록 행동을 선택해야 한다.

정책은 에이전트가 상황에 따라 어떻게 행동을 해야 할지 확률적으로 나타낸 기준이다.

$$\pi(a|s) = P(A_t=a|S_t=s)$$

### 12.2.5 가치 함수

가치 함수는 주어진 정책 아래에서 특정 상태인 s에서부터 앞으로 얻을 수 있는 보상의 누적 총합의 기댓값을 표현한다.

$$v_\pi(s) \doteq \mathbb E_\pi [\sum_{k=0}^\infty \gamma^k R_{t+k+1}|S_t=s], \forall s \in \mathcal S$$

#### 행동 가치 함수

- 큐 함수(Q-function)이라고도 함
- 주어진 정책 아래 특정 시점의 상황에서 행동을 선택했을 때 앞으로 얻을 수 있는 보상의 누적 합의 기댓값

$$q_\pi(s,a) \doteq \mathbb E_\pi [\sum_{k=0}^\infty \gamma^k R_{t+k+1} | S_t = s, A_t = a]$$

### 12.2.6 벨만 방정식

가치 함수와 행동 가치 함수를 벨만 방정식의 형태로 나타낼 수 있다. 벨만 방정식으로 나타낼 수 있다는 것은 동적 프로그래밍 알고리즘 문제로 접근할 수 있다는 뜻이다. 즉, 단순히 최단경로를 찾는 문제와 똑같이 생각할 수 있다.

$$\begin{align}
v_*(s) & = \max_a \mathbb E [R_{t+1} + \gamma v_*(S_{t+1}) | S_t = s, A_t = a] \\
& = \max_a \sum_{s', r} P(s',r|s,a)[r + \gamma v_*(s')]\\
\end{align} $$

$$\begin{align}
q_*(s) & = \mathbb E [R_{t+1} + \gamma \max_{a'} q_*(S_{t+1}, a') | S_t = s, A_t = a] \\
& = \sum_{s', r} P(s',r|s,a)[r + \gamma \max_a q_*(s', a')]\\
\end{align} $$

### 12.2.7 몬테카를로 방법

대부분의 경우 $P(s', r|s, a)$을 모른다. 따라서 시뮬레이션 등의 경험을 통해 에이전트를 학습해야 한다.

$$V(S') \leftarrow V(S_t) + \alpha [G_t - V(S_t)]$$

몬테카를로 방법처럼 샘플링을 통해 벨만의 기대 방정식을 해결할 수 있다. 문제는 긴 에피소드일 경우 에피소드가 끝나야 $G_t$를 구할 수 있기 때문에 시간과 자원이 많이 든다.

### 12.2.8 시간차 학습

에피소드 보상의 누적 합 없이도 바로 가치 함수를 업데이트할 수 있다.

$$V(S_t) \leftarrow V(S_t) + \alpha [R_{t+1} + \gamma V(S_{t+1}) - V(S_t)]$$

#### 큐 러닝

올바른 행동 가치 함수를 알고 있다면, 항상 기대누적보상을 최대화하는 선택을 할 수 있다. 이때 행동 가치 함수를 잘 학습하는 것을 큐 러닝이라고 한다. 타깃과 현재 가치 함수 사이의 차이를 줄이면 옯른 큐 함수를 학습하게 된다.

$$Q(S_t. A_t) \leftarrow Q_(S_t, A_t) + \alpha [\overbrace{R_{t+1} + \gamma max_a Q(S_{t+1}, a)}^{Target} - \overbrace{Q(S_t, A_t)}^{Current}]$$

#### 딥-큐 러닝

큐 함수를 배울 때 상태 공간과 행동 공간의 크기가 너무 커서 상황과 행동이 희소할 경우 문제가 생긴다. 상황과 행동이 불연속적인 별개의 값이라도, 큐 함수를 근사하면 문제를 해결할 수 있다. 이를 딥-큐 러닝(DQN)이라고 한다.

$$Q(S_t,A_t) \leftarrow \underbrace{Q(S_t, A_t)}_{Approximated} + \alpha [R_{t+1} + \gamma max_a Q(S_{t+1},a) - Q(S_t,A_t)]$$

## 12.3 정책 기반 강화학습

### 12.3.1 폴리시 그래디언트

- 가치 기반 강화학습: 인공신경망을 사용해 어떤 행동을 선택했을 때 얻을 수 있는 보상을 예측하도록 훈련, 행동의 선택이 확률적으로 나오지 않음
- 정책 기반 강화학습: 인공신경망의 행동에 대한 보상을 역전파 알고리즘을 통해 전달하여 학습, 확률적인 과정 거침
- 폴리시 그래디언트 수식 $\pi_\theta(a|s) = P_\theta(a|s) = P(a|s;\theta)$
    - 신경망 가중치 파라미터 $\theta$는 현재 상태 $s$가 주어졌을 때, 어떤 행동 $a$을 선택해야 하는지에 대한 확률 반환

$$\begin{align}
J(\theta) & = \mathbb E_{\pi_\theta} [r] = v_\theta(s_0) \\
& = \sum_{s in S} d(s) \sum_{a in A} \pi_\theta(s, a) \mathcal R_{s,a} \\
\end{align}$$

- 경사상승법 사용

$$\nabla_\theta J(\theta)  = \mathbb E_{\pi_\theta} [\nabla_\theta log \pi_\theta (a|s) r]$$

즉각적인 보상 r 대신 큐 함수를 사용할 수도 있다.

$$\nabla_\theta J(\theta)  = \mathbb E_{\pi_\theta} [\nabla_\theta log \pi_\theta (a|s) Q^{\pi_\theta(s,a)}]$$

폴리시 그래디언트의 신경망에 대해서는 미분을 계산해야 하지만, 큐 함수에 대해서는 미분할 필요가 없다. 따라서 어떠한 함수라도 보상 함수로 사용할 수 있다.

폴리시 그래디언트는 기존 경사도의 방향에 스칼라 값을 곱해주므로 실제 보상을 최대화하는 직접적인 방향을 지정해줄 수는 없다. 따라서 최적의 방향을 스스로 찾아갈 수 없으므로, 훈련이 어렵다.

### 12.3.2 MLE vs 폴리시 그래디언트

## 12.5 강화학습을 활용한 지도학습

### 12.5.1 리스크 최소화 훈련

$$\begin{align}
\mathcal R (\theta) & = \sum_{s=1}^S \mathbb E_{y|x^{(s)}; \theta} [\vartriangle (y, y^{(s)})] \\
& = \sum_{s=1}^S \sum_{y \in \mathcal Y(x^{(s)})} P(y|x^{(s)};\theta) \vartriangle(y, y^{(s)}) \\
\end{align}$$

- $\mathcal Y(x^{(s)})$는 탐색 공간의 전체 집합, S번째 입력이 주어졌을 때 가능한 정답의 집합
- $\vartriangle(y, y^{(s)})$는 입력과 파라미터가 주어졌을 때, 샘플링한 $y$와 실제 정답 $y^{(s)}$과의 차이
- 리스크는 주어진 입력과 현재 파라미터 상에서 얻은 $y$를 통해 현재 모델을 구하고, 동시에 리스크의 기댓값을 구한다고 볼 수 있다.

$$\DeclareMathOperator*{\argmin}{argmin}
\DeclareMathOperator*{\argmax}{argmax}
\hat \theta_{MRT} = \ \argmin_{\theta} \ \mathcal R(\theta)$$

리스크를 최소화하는 것은 보상을 최대화하는 것과 동일한 의미이다. 실제 구현할 때는 보상함수 BLEU 점수에 -1을 곱하여 리스크 함수를 만든다. 전체 공간을 탐색할 수 없으므로 서브스페이스에서 샘플링한다.

#### MRT 최종 수식 해석

1. S번째 입력 $x^{(s)}$를 신경망 $\theta$에 넣어 얻은 로그확률 logP(y|x^{(s)};\theta)를 $\theta$에 대해 미분하여 그래디언트를 얻어낸다.
2. $\theta$로부터 샘플링한 $y$와 실제 정답 $y^{(s)}$와의 차이값에서, 또 다시 $\theta$로부터 샘플링하여 얻은 $y$와 실제 정답 $y^{(s)}$와의 차이의 기댓값을 빼 준 값을 구한다.
3. 그 값을 리스크로 방금 계산한 로그 확률값의 그래디언트에 곱해준다.
4. 이 과정을 전체 데이터셋 $S$에 대해 수행한 후에 합을 구하고 학습률 $\alpha$를 곱한다.

#### 구현

1. 주어진 입력 문장에 대해 정책$\theta$를 이용하여 번역 문장을 샘플링한다.
2. 샘플링 문장과 정답 문장 사이의 BLEU를 계산하고 -1을 곱해 리스크로 변환한다.
3. 로그 확률 분포 전체에 리스크를 곱해주고 각 샘플과 time-step별로 구해진 음의 로그 확률값의 합에 -1을 곱해준다.
4. 로그 확률값의 합에 $\theta$로 미분을 수행하면, 역전파를 통해서 신경망 $\theta$ 전체에 그래디언트가 구해진다.
5. 이미 리스크를 확률 분포에 곱했으므로, 곧바로 이 그래디언트를 통해 경사하강법을 수행하여 최적화한다.

#### 파이토치 예제 코드

GLUE 함수 사용

In [ ]:
from tqdm import tqdm
from nltk.translate.gleu_score import sentence_gleu as score_func

import torch
import torch.nn.utils as torch_utils

import utils
import data_loader

from simple_nmt.trainer import Trainer

In [2]:
def _get_reward(self, y_hat, y, n_gram=6):
    
    # |y| = (batch_size, length1)
    # |y_hat| = (batch_size, length21)
    
    scores = []
    
    for b in range(y.size(0)):
        ref = []
        hyp = []
        for t in range(y.size(1)):
            ref += [str(int(y[b, t]))]
            if y[b, t] == data_loader.EOS:
                break
        
        for t in range(y_hat.size(1)):
            hyp += [str(int(y_hat[b, t]))]
            if y_hat[b, t] == data_loader.EOS:
                break
        
        scores += [score_func([ref], hyp, max_len=n_gram) * 100.]
        
    scores = torch.FloatTensor(scores).to(y.device)
    
    return socres

In [3]:
def _get_gradient(self, y_hat, y, crit=None, reward=1):
    # |y| = (batch_size, length)
    # |y_hat| = (batch_size, length, output_size)
    # |reward| = (batch_size)
    crit = self.crit if crit is None else crit
    
    y_hat = y_hat * -reward.view(-1, 1, 1).expand(*y_hat.size())
    
    log_prob = -crit(y_hat.contiguous().view(-1, y_hat.size(-1)),
                     y.contiguous().view(-1)
                     )
    log_prob.div(y.size(0)).backward()
    
    return log_prob

몬테카를로 샘플링을 통해 베이스라인 보상값을 근사한다. K가 클수록 정확하지만 1로 해도 잘 작동한다.

In [ ]:
def train_epoch(self,
                train,
                optimizer,
                max_grad_norm=5,
                verbose=VERBOSE_SILENT
                ):
    total_reward, total_actor_reward = 0, 0
    total_grad_norm = 0
    avg_reward, avg_actor_reward = 0, 0
    avg_param_norm, avg_grad_norm = 0, 0
    sample_cnt = 0
    
    progress_bar = tqdm(train,
                        desc='Training: ',
                        unit='batch'
                        ) if verbose is VERBOSE_BATCH_WISE else train
    
    for idx, mini_batch in enumerate(progress_bar):
        x, y = mini_batch.src, mini_batch.tgt[0][:, 1:]
        # |x| = (batch_size, length)
        # |y| = (batch_size, length)
        
        optimizer.zero_grad()
        
        y_hat, indice = self.model.search(x, 
                                          is_greedy=False,
                                          max_length=self.config.max_length
                                          )
        
        actor_reward = self._get_reward(indice,
                                        y,
                                        n_gram=self.config.rl_n_gram
                                        )
        
        baseline = []
        with torch.no_grad():
            for i in range(self.config.n_samples):
                _, sampled_indice = self.model.search(x,
                                                      is_greedy=False,
                                                      max_length=self.config.max_length
                                                      )
                baseline += [self._get_reward(sampled_indice,
                                               y,
                                               n_gram=self.config.rl_n_gram
                                               )]
                baseline = torch.stack(baseline).sum(dim=0).div(self.config.n_samples)
                
            final_reward = actor_reward - baseline
            
            self._get_gradient(y_hat, indice, reward=final_reward)
            
            total_reward += float(final_reward.sum())
            total_actor_reward += float(actor_reward.sum())
            sample_cnt += int(actor_reward.size(0))
            total_grad_norm += float(utils.get_grad_norm(self.model.parameters()))
            
            avg_reward = total_reward / sample_cnt
            avg_actor_reward = total_actor_reward / sample_cnt
            avg_grad_norm = total_grad_norm / (idx + 1)
            
            if verbose is VERBOSE_BATCH_WISE:
                progress_bar.set_postfix_str('|g_param|=%.2f rwd=%4.2f avg_frwd=%.2e BLEU=%.4f' % (avg_grad_norm,
                                                                                                   float(actor_reward.sum().div(y.size(0))),
                                                                                                   avg_reward,
                                                                                                   avg_actor_reward))
                
            torch_utils.clip_grad_norm_(self.model.parameters(),
                                        self.config.max_grad_norm
                                        )
            optimizer.step()
            
            if idx >= len(progress_bar) * self.config.train_ratio_per_epoch:
                break
            
        if verbose is VERBOSE_BATCH_WISE:
            progress_bar.close()
        
        return avg_actor_reward, param_norm, avg_grad_norm

## 12.6 강화학습을 활용한 비지도 학습

### 12.6.1 비지도학습을 통한 NMT

단일 언어 코퍼스만을 사용해서 번역기 제작 방법 제안. 언어에 상관없이 같은 의미의 문장일 경우 인코더가 같은 값으로 임베딩할 수 있도록 훈련. 인코딩된 문장의 언어를 맞추는 판별자가 필요, 인코더는 판별자를 속이도록 훈련. 언어에 상관없이 1개씩의 인코더와 디코더 사용

#### 디노이징 오토인코더

오토인코더에게 단순히 복사 작업을 지시하는 대신, 노이즈를 섞어준 소스 문장에서 노이즈를 제거하면서 입력값을 복원할 수 있도록 훈련

$$\mathcal L_{auto}(\theta_{enc}, \theta_{dec}, \mathcal Z, l) = \mathbb E_{x \sim \mathcal D_l, \hat x \sim d(e(C(x), l), l)}[\vartriangle(\hat x, x)]$$

노이즈 모델 C(x)는 임의로 문장 내 단어들을 드롭하거나 순서를 섞는 역할

#### 크로스 도메인 훈련

저성능의 번역 모델에서 언어(l_2)의 노이즈가 추가되어 번역된 문장을 다시 언어(l_1) 소스 문장으로 원상 복구하는 작업

#### 적대적 학습

인코더가 언어와 상관없이 항상 같은 분포로 장재 공간에 문장 벡터를 임베딩하는지 감시하는 판별자가 추가되어 적대적 학습 진행